In [38]:
'''
This file reads json file in project/hand_pose/Annotation.json and output project/hand_pose/hand_labels_r90.csv
'''

'\nThis file reads json file in project/hand_pose/Annotation.json and output project/hand_pose/hand_labels_r90.csv\n'

In [39]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import json
from pprint import pprint
from matplotlib import pyplot as plt
from PIL import Image
import sys
import numpy as np
import matplotlib.patches as patches
import cv2
import pandas as pd
import warnings
warnings.simplefilter("ignore")

# Get 2500 images for rotate 90

In [40]:
base_dir = "../hand_pose"
image_dir = os.path.join(base_dir, 'Color')

In [41]:
img_list = os.listdir(image_dir)
img_name = [img[:-4] for img in img_list]

In [42]:
'''
def rotate(image, theta):
    (h, w) = image.shape[:2]
    center = (w / 2, h / 2)
    M = cv2.getRotationMatrix2D(center, theta, 1)
    rotated = cv2.warpAffine(image, M, (int(w),int(h)), cv2.INTER_LINEAR,
                             borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))
    return rotated
'''

'\ndef rotate(image, theta):\n    (h, w) = image.shape[:2]\n    center = (w / 2, h / 2)\n    M = cv2.getRotationMatrix2D(center, theta, 1)\n    rotated = cv2.warpAffine(image, M, (int(w),int(h)), cv2.INTER_LINEAR,\n                             borderMode=cv2.BORDER_CONSTANT, borderValue=(255, 255, 255))\n    return rotated\n'

In [43]:
image = cv2.imread(os.path.join(image_dir,img_list[1]))
(h, w) = image.shape[:2]
print(h, w)

1080 1920


In [44]:
img_amount = len(img_list)
print(img_amount)

111


In [45]:
# Following code use tiny dataset for script test

In [46]:
target_dir = os.path.join(base_dir, 'Rotate_90_imgs')
for i in range(1, img_amount, 10):
    img = cv2.imread(os.path.join(image_dir,img_list[i]))
    cv2.imwrite(os.path.join(target_dir, img_name[i] +'_r90.jpg'), np.rot90(img))
    #cv2.imwrite(os.path.join(target_dir, img_name[i] +'_r90.jpg'), rotate(img, 90))

In [47]:
# Following code use real hand_pose dataset with folder "Color" from assignment 1&3

In [48]:
'''
target_dir = os.path.join(base_dir, 'Rotate_90_imgs')
for i in range(1, 56000, 22):
    img = cv2.imread(os.path.join(image_dir,img_list[i]))
    cv2.imwrite(os.path.join(target_dir, img_name[i] +'_r90.jpg'), np.rot90(img))
    #cv2.imwrite(os.path.join(target_dir, img_name[i] +'_r90.jpg'), rotate(img, 90))
'''

"\ntarget_dir = os.path.join(base_dir, 'Rotate_90_imgs')\nfor i in range(1, 56000, 22):\n    img = cv2.imread(os.path.join(image_dir,img_list[i]))\n    cv2.imwrite(os.path.join(target_dir, img_name[i] +'_r90.jpg'), np.rot90(img))\n    #cv2.imwrite(os.path.join(target_dir, img_name[i] +'_r90.jpg'), rotate(img, 90))\n"

In [49]:
print(img)

[[[123 140 136]
  [123 140 136]
  [123 140 136]
  ...
  [138 158 146]
  [138 158 146]
  [138 158 146]]

 [[123 140 136]
  [123 140 136]
  [123 140 136]
  ...
  [138 158 146]
  [138 158 146]
  [138 158 146]]

 [[123 140 136]
  [123 140 136]
  [123 140 136]
  ...
  [138 158 146]
  [138 158 146]
  [138 158 146]]

 ...

 [[ 65  86  88]
  [ 73  94  96]
  [101 122 124]
  ...
  [ 97 128 125]
  [ 96 127 124]
  [ 95 126 123]]

 [[ 67  88  90]
  [ 73  94  96]
  [100 121 123]
  ...
  [ 98 129 126]
  [ 97 128 125]
  [ 97 128 125]]

 [[ 69  90  92]
  [ 74  95  97]
  [101 122 124]
  ...
  [100 131 128]
  [ 99 130 127]
  [ 98 129 126]]]


In [50]:
print(np.rot90(img))

[[[138 158 146]
  [138 158 146]
  [138 158 146]
  ...
  [ 95 126 123]
  [ 97 128 125]
  [ 98 129 126]]

 [[138 158 146]
  [138 158 146]
  [138 158 146]
  ...
  [ 96 127 124]
  [ 97 128 125]
  [ 99 130 127]]

 [[138 158 146]
  [138 158 146]
  [138 158 146]
  ...
  [ 97 128 125]
  [ 98 129 126]
  [100 131 128]]

 ...

 [[123 140 136]
  [123 140 136]
  [123 140 136]
  ...
  [101 122 124]
  [100 121 123]
  [101 122 124]]

 [[123 140 136]
  [123 140 136]
  [123 140 136]
  ...
  [ 73  94  96]
  [ 73  94  96]
  [ 74  95  97]]

 [[123 140 136]
  [123 140 136]
  [123 140 136]
  ...
  [ 65  86  88]
  [ 67  88  90]
  [ 69  90  92]]]


# json to jsv for 2500 rotated images

In [51]:
tensorflow_path = "phase1_models"
sys.path.append(tensorflow_path + "/research")
sys.path.append(tensorflow_path + "/research/object_detection")
sys.path.append(tensorflow_path + "/research/object_detection/utils")
from utils import label_map_util
from utils import visualization_utils as vis_util

In [52]:
base_dir = "../hand_pose"
json_file = os.path.join(base_dir, 'annotation.json')
segment_length = 20
csv_file_r90 = os.path.join(base_dir, 'hand_labels_r90.csv')
image_dir_r90 = os.path.join(base_dir, 'Rotate_90_imgs')

In [53]:
img_list_r90 = os.listdir(image_dir_r90)

In [54]:
img_name_r90 = [img[:-8] for img in img_list_r90]

In [55]:
with open(json_file) as f:
    data = json.load(f)
pprint(f)

<_io.TextIOWrapper name='../hand_pose/annotation.json' mode='r' encoding='UTF-8'>


In [56]:
len(data)

94126

In [57]:
print(img_name_r90)

['000_2531', '000_2567', '000_2195', '000_2351', '000_2453', '000_2465', '000_2495', '000_2093', '000_2237', '000_2099', '000_2561']


In [58]:
len(img_list_r90)

11

In [59]:
def paint_boundary_r90(index):
    img = cv2.imread(os.path.join(image_dir_r90,img_list_r90[index]))
    if img_name_r90[index]+"_L" in data:
        pos = record_boundary_r90(index, "_L")
        img = cv2.rectangle(img, pos[0], pos[1] , (0, 255, 0), 3) #(row['xmin'], row['ymin']), (row['xmax'], row['ymax'])
    if img_name[index]+"_R" in data:
        pos = record_boundary_r90(index, "_R")
        img = cv2.rectangle(img, pos[0], pos[1] , (0, 255, 0), 3) #(row['xmin'], row['ymin']), (row['xmax'], row['ymax'])     
    return img

In [60]:
def record_boundary_r90(index, direction):
    name = img_name_r90[index]+direction
    xy = np.array(data[name])
    x_arr = xy[:,0]
    y_arr = xy[:,1]
    segment_length = 20
    x_max = max(y_arr) + segment_length
    y_max = max(w - x_arr) + segment_length
    x_min = min(y_arr) - segment_length
    y_min = min(w - x_arr) - segment_length
    
    '''
    x_max = max(x_arr) + segment_length
    y_max = max(y_arr) + segment_length
    x_min = min(x_arr) - segment_length
    y_min = min(y_arr) - segment_length
    '''
    return [(int(x_min), int(y_min)), (int(x_max), int(y_max))]

In [61]:
def return_value_r90(index, direction):
    pos = record_boundary_r90(index, direction)
    xmin = pos[0][0]
    ymin = pos[0][1]
    xmax = pos[1][0]
    ymax = pos[1][1]
    value = (img_name_r90[index]+'_r90.jpg', #+direction,
             xmax-xmin,
             ymax-ymin,
             'hand',
             xmin,
             ymin,
             xmax,
             ymax
             )
    return value

In [62]:
xml_list_r90 = []
#for xml_file in glob.glob(path + '/*.xml'):
for index, img in enumerate(img_name_r90):
    if img + '_R' in data:
        xml_list_r90.append(return_value_r90(index, '_R'))
    if img + '_L' in data:
        xml_list_r90.append(return_value_r90(index, '_L'))
column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
xml_df_r90 = pd.DataFrame(xml_list_r90, columns=column_name) #use panda
#print(xml_df)

In [63]:
len(xml_list_r90)

17

In [64]:
xml_df_r90.to_csv(csv_file_r90, index=None)
print('Successfully converted xml to csv.')

Successfully converted xml to csv.


# Append 2500 images csv to hand_labels.csv

In [65]:
base_dir = "../hand_pose"
csv_file_r90 = os.path.join(base_dir, 'hand_labels_r90.csv')
csv_file = os.path.join(base_dir, 'hand_labels.csv')

In [66]:
labels_r90 = pd.read_csv(csv_file_r90)

In [67]:
labels = pd.read_csv(csv_file)

In [68]:
len(labels_r90)

17

In [69]:
len(labels)

233

In [70]:
results = pd.concat([labels,labels_r90])
results.to_csv(os.path.join(base_dir, 'hand_labels.csv'))

In [71]:
len(results)

250